In [1]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import itertools

os.chdir('/Users/yjp/capp30254/crypto-currency/models')

In [2]:
data = pd.read_csv("bitcoin_train.csv")
cols = data.columns
#for i,c in enumerate(cols):
#    print(i, c)

rv = []
cols_list = list(range(7, 34))

for l in range(1, len(cols_list) + 1):
    rv.append(list(itertools.combinations(cols_list, l)))
    print('we are at', l)  

we are at 1
we are at 2
we are at 3
we are at 4
we are at 5
we are at 6
we are at 7
we are at 8
we are at 9
we are at 10
we are at 11
we are at 12
we are at 13
we are at 14
we are at 15
we are at 16
we are at 17
we are at 18
we are at 19
we are at 20
we are at 21
we are at 22
we are at 23
we are at 24
we are at 25
we are at 26
we are at 27


In [5]:
print(rv)

In [2]:
#X = pd.read_csv("merged_datasets.csv",usecols=[5, 16, 18, 20, 23, 30, 31, 32])
#y = pd.read_csv("merged_datasets.csv",usecols=[9])
#X['compound_weighted_replies'] = X['compound_weighted_replies'].fillna(0)

#X = X.to_numpy()
#y = y.to_numpy()

X = pd.read_csv("bitcoin_train.csv",usecols=[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 24, 27, 28, 29, 32, 33, 34])
y = pd.read_csv("bitcoin_train.csv",usecols=[35])

X = X.to_numpy()
y = y.to_numpy()

FileNotFoundError: [Errno 2] No such file or directory: 'merged_datasets.csv'

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
y_train = y_train.reshape(-1)

In [5]:
lr = LogisticRegression()

In [6]:
lr.fit(X_train, y_train)
y_hat=lr.predict(X_test)

In [7]:
confusion = metrics.confusion_matrix(y_test, y_hat)

In [8]:
accuracy = np.sum(np.diagonal(confusion))/len(y_test)
print(accuracy)

0.5417721518987342
